In [2]:
import pandas as pd

In [3]:
boroughs = [
    'brooklyn', 'bronx', 'manhattan', 'queens', 'staten_island'
]

stops = pd.concat(
    [pd.read_csv(f'./data/{borough}/stops.txt') for borough in boroughs],
    ignore_index=True
)

stop_times = pd.concat(
    [pd.read_csv(f'./data/{borough}/stop_times.txt') for borough in boroughs],
    ignore_index=True
)

trips = pd.concat(
    [pd.read_csv(f'./data/{borough}/trips.txt') for borough in boroughs],
    ignore_index=True
)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [18]:
stops.size

115209

In [4]:
combined = pd.merge(stop_times, trips, on='trip_id', how='left')
stop_and_route = combined[['stop_id', 'route_id']].drop_duplicates().merge(stops, how='left', on='stop_id')

In [4]:
# how many bus stops of each type
stop_and_route.groupby('stop_id')['route_id'].nunique().value_counts()

1     8486
2     2676
3      823
4      280
5      130
6       73
7       38
8       36
9       28
10      10
11       7
12       3
13       3
15       2
17       1
14       1
16       1
Name: route_id, dtype: int64

In [21]:
len(stop_and_route['stop_id'].unique())

12598

In [23]:
stop_and_route['route_id'].unique().size

242

In [25]:
stop_and_route.head()

,stop_id,route_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,303345,B42,ROCKAWAY PKY SUBWAY/ROCKAWAY PKY SUBWAY,NaN,40.645845,-73.902462,NaN,NaN,0,NaN
1,308044,B42,ROCKAWAY PKY/GLENWOOD RD,NaN,40.644964,-73.902255,NaN,NaN,0,NaN
2,306590,B42,ROCKAWAY PY/FLATLANDS AV,NaN,40.643078,-73.900179,NaN,NaN,0,NaN
3,303348,B42,ROCKAWAY PY/AV J,NaN,40.641660,-73.898593,NaN,NaN,0,NaN
4,303349,B42,ROCKAWAY PY/AV K,NaN,40.640465,-73.897268,NaN,NaN,0,NaN


In [38]:
stop_to_all_busses = stop_and_route.groupby('stop_id')['route_id'].apply(lambda x: list(set(x)))

In [17]:
len(stop_and_route[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']].drop_duplicates(subset=['stop_id']))

12598

In [39]:
combined = pd.merge(
    stop_and_route[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']].drop_duplicates(subset=['stop_id']),
    stop_to_all_busses,
    on='stop_id'
).rename(columns={'route_id': 'routes'})

In [40]:
combined['route_count'] = combined['routes'].apply(len)

In [41]:
combined[combined['route_count'] > 10]

,stop_id,stop_name,stop_lat,stop_lon,routes,route_count
4610,805039,Gowanus Exp./22 St (N.B.),40.663351,-73.998119,"[SIM5, X27, SIM4C, X37, SIM34, X38, X28, SIM35...",12
4613,450402,CHURCH ST/FULTON ST,40.711727,-74.010261,"[X27, SIM3C, SIM1, SIM4C, SIM4X, SIM34, SIM1C,...",13
4660,402130,E 23 ST/BROADWAY,40.741142,-73.988790,"[M23+, X27, SIM4C, X37, X64, X68, X63, X38, X2...",12
4691,402143,E 23 ST/1 AV,40.737297,-73.979360,"[M23+, SIM11, X37, X64, X68, X63, X38, SIM6, S...",11
7520,400070,TRINITY PL/RECTOR ST,40.708007,-74.012714,"[SIM3C, SIM4C, SIM1, SIM4X, SIM34, M55, SIM1C,...",12
7525,405024,CHURCH ST/WORTH ST,40.716678,-74.006421,"[SIM3C, SIM4C, SIM1, SIM4X, SIM34, M55, SIM1C,...",11
7604,402214,E 57 ST/LEXINGTON AV,40.760727,-73.968725,"[SIM8X, SIM11, SIM26, M31, X64, X68, M57, X63,...",13
7941,400359,MADISON AV/E 47 ST,40.756492,-73.976474,"[SIM8X, Q32, SIM11, M3, M4, M1, SIM8, M2, SIM6...",11
10234,404300,MADISON AV/E 54 ST,40.760615,-73.973461,"[SIM8X, SIM11, SIM4C, X64, X68, X63, SIM8, SIM...",11
10743,805033,Verrazano Bridge/Lee Av (N.B.),40.610786,-74.031356,"[SIM5, SIM4C, SIM4X, SIM34, SIM33, SIM3C, SIM3...",13


In [43]:
combined.to_csv('cleaned.csv')

In [45]:
combined.to_json('cleaned.json', orient='records')

In [7]:
stop_and_route['route_id'].unique()

array(['B42', 'B82', 'B25', 'B17', 'Q24', 'B12', 'B14', 'Q56', 'B15',
       'B83', 'B65', 'B45', 'B91A', 'B84', 'B82+', 'B41', 'B44', 'B46',
       'B49', 'B31', 'B2', 'B44+', 'B46+', 'B1', 'B47', 'Q54', 'B48',
       'B54', 'Q55', 'Q58', 'B52', 'B60', 'B38', 'B26', 'B13', 'B57',
       'B7', 'B20', 'B62', 'Q59', 'B24', 'B32', 'B39', 'B8', 'B61', 'B35',
       'B68', 'B63', 'B43', 'B11', 'B70', 'B67', 'B16', 'B4', 'B9', 'B37',
       'B69', 'B36', 'B6', 'B3', 'B64', 'X27', 'X28', 'B74', 'X38', 'X37',
       'BX41', 'BX39', 'BX41+', 'BX12+', 'BX26', 'BX22', 'BX40', 'BX42',
       'BX5', 'BX29', 'BX4', 'BX4A', 'BX34', 'BX12', 'BX24', 'BX16',
       'BX28', 'BX38', 'BX30', 'BX15', 'M100', 'BX7', 'BX20', 'BX10',
       'BX1', 'BX2', 'BX9', 'BX13', 'BX18', 'BX3', 'BX6+', 'BX33', 'BX8',
       'BX17', 'BX21', 'BX36', 'BX35', 'BX46', 'BX19', 'BX31', 'BX32',
       'BX6', 'BX27', 'BX11', 'M14A', 'M14D', 'M60+', 'M9', 'M42', 'M50',
       'M86+', 'M20', 'M34+', 'M34A+', 'M66', 'M23+', 'M22', '